In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
import numpy as np
from scipy.stats import norm
import sys

In [0]:
CURRENT_D            = current_date()
CURRENT_DT           = current_timestamp()

In [0]:
SERVICE_LEVEL = 0.90
z = np.round(norm.ppf(SERVICE_LEVEL), 2)
 

In [0]:
MATERIAL_LIST = spark.read.table('default.sample_material_list').select('MATNR','LGORT','AREA')

PLANT_LIST = spark.read.table('default.plant_list').select('PLANT','LGORT','AREA')
PLANTS_TABLE         =   PLANT_LIST.select("PLANT").distinct()
# display(PLANTS_TABLE)

In [0]:
LEADTIME = spark.read.table("default.leadtime").withColumn("MATNR", F.substring("MATERIAL", -7, 7))

INVENTORY = spark.read.table('default.inventory').withColumnRenamed("NEW_MATC","MATNR")\
  .withColumnRenamed('STORAGE_BIN',"LGORT").drop_duplicates()

OPEN_REPLEN = spark.read.table('default.open_po').withColumnRenamed("NEWMATERIAL","MATNR")\
  .withColumnRenamed('STORAGE',"LGORT").drop_duplicates()

BO_Details = spark.read.table('default.backorder').withColumnRenamed('MATCODE','MATNR')

MATERIAL_MASTER = spark.read.table('default.material_master')


In [0]:
matnr_cons = spark.read.table('default.matnr_cons_df')
demand_stats = matnr_cons.groupby('MATNR','AREA','LGORT').agg(round(stddev('y'),2).alias('STDDEV_DEMAND'),
                                               round(avg('y'),2).alias('demand_per_day'))

display(demand_stats)

MATNR,AREA,LGORT,STDDEV_DEMAND,demand_per_day
1223406,NPI,1000,2.12,1.2
1415244,NPI,1000,3.73,2.88
1166284,NPI,1000,16.88,9.28
1415319,NPI,1000,11.03,8.51
1269875,NPI,1000,2.73,1.86
1269877,NPI,1000,3.24,2.49
1426657,NPI,1000,6.49,6.02
1415603,NPI,1000,10.77,10.16
1426745,NPI,1000,10.54,10.16
1269878,NPI,1000,3.4,2.55


In [0]:
lead_time    =   LEADTIME.join(PLANTS_TABLE,['PLANT'],'inner').filter(F.col("REQ_TYPE") == "ZINV")\
                             .select(LEADTIME.columns)

lead_time_stats = lead_time.groupby("MATNR")\
                                     .agg(F.floor(F.expr('percentile(DAYS, 0.5)')).alias("MEDIAN_LEADTIME"), 
                                           round( F.stddev(col('DAYS')),0).alias('std'))\
                                     .join(MATERIAL_LIST.select("MATNR",'AREA',"LGORT"),["MATNR"],"inner")

display(lead_time_stats)

MATNR,MEDIAN_LEADTIME,std,AREA,LGORT
1269878,49,23.0,NPI,1000
1426745,181,62.0,NPI,1000
1415244,182,132.0,NPI,1000
1415319,176,103.0,NPI,1000
1223406,144,41.0,NPI,1000
1415603,167,56.0,NPI,1000
1426657,286,146.0,NPI,1000
1166284,44,26.0,NPI,1000
1269875,47,24.0,NPI,1000
1269877,52,21.0,NPI,1000


In [0]:
inventory_stats = INVENTORY.join(PLANT_LIST,['PLANT','LGORT'],'inner')\
                           .join(MATERIAL_LIST,['MATNR','AREA','LGORT'],'inner')\
                             .groupby('MATNR','AREA','LGORT').agg(sum('QUANTITY').alias('Inventory'),
                                                                  sum('TOTAL_VALUATION').alias('INV_CAPEX') ) 
# display(inventory_stats)

In [0]:
open_po = OPEN_REPLEN.drop('AREA').join(PLANT_LIST,['PLANT','LGORT'],'inner')\
                           .join(MATERIAL_LIST,['MATNR','AREA','LGORT'],'inner')\
                             .groupby('MATNR','AREA','LGORT').agg(sum('OPEN_QTY').alias('Open_POs') ) 

display(open_po)

MATNR,AREA,LGORT,Open_POs
1223406,NPI,1000,251
1415244,NPI,1000,122
1415319,NPI,1000,157
1269875,NPI,1000,3
1269877,NPI,1000,5
1426657,NPI,1000,1268
1415603,NPI,1000,1599
1426745,NPI,1000,1610


In [0]:
pricing = MATERIAL_MASTER.select('MATNR', col('CURRENT_PRICE').alias('Unit_Price'))\
                         .join(MATERIAL_LIST,['MATNR'],'inner')
# display(pricing)

In [0]:
demand_over_leadtime = demand_stats.select("MATNR","AREA","LGORT","demand_per_day","STDDEV_DEMAND")\
                .join(lead_time_stats.select("MATNR","AREA","LGORT","MEDIAN_LEADTIME"),
                                         ["MATNR","AREA","LGORT"], how="inner")\
                                   .join(inventory_stats.select("MATNR","AREA","LGORT" ,"Inventory", "INV_CAPEX"),["MATNR","AREA","LGORT"], how="left")\
                                   .join(open_po,["MATNR","AREA","LGORT"], how="left")\
                                   .join(pricing,["MATNR","AREA","LGORT"], "left").fillna(0)

demand_over_leadtime_selected = demand_over_leadtime.withColumn("mean_demand_over_leadtime", 
                                                       F.round(F.col("MEDIAN_LEADTIME")*F.col("demand_per_day"), 2))\
                       .withColumn("std_demand_over_leadtime", F.round(F.sqrt(F.col("MEDIAN_LEADTIME"))*F.col("STDDEV_DEMAND"), 2))\
                       .withColumn("Reorder_Point", F.col("mean_demand_over_leadtime") + z*F.col("std_demand_over_leadtime"))\
                       .withColumn("Reorder_Point", F.when(F.col('Reorder_Point')== 0,1).otherwise(F.ceil(col('Reorder_Point'))))\
                        .select("MATNR","AREA","LGORT","demand_per_day","MEDIAN_LEADTIME"
                        ,"Inventory","INV_CAPEX","Open_POs", "Unit_Price","Reorder_Point")\
                      .withColumn("Inventory_Exhaust_Days",F.round((F.col("Inventory"))/(F.col("demand_per_day")))\
                                  .cast(T.IntegerType()))\
                      .withColumn("Inventory_Exhaust_Date", F.expr("date_add(CURRENT_DATE(), Inventory_Exhaust_Days)"))\
                      .drop("today", "Inventory_Exhaust_Days","demand_per_day_6month","demand_per_day_12month")

display(demand_over_leadtime_selected)

MATNR,AREA,LGORT,demand_per_day,MEDIAN_LEADTIME,Inventory,INV_CAPEX,Open_POs,Unit_Price,Reorder_Point,Inventory_Exhaust_Date
1269878,NPI,1000,2.55,49,0,0.0,0,80.0,156,2023-10-12
1426745,NPI,1000,10.16,181,3690,88630.54,1610,52.0,2021,2024-10-09
1415244,NPI,1000,2.88,182,32,12566.4,122,341.0,589,2023-10-23
1415319,NPI,1000,8.51,176,189,9792.09,157,49.0,1686,2023-11-03
1223406,NPI,1000,1.2,144,41,733.0799999999999,251,16.16,206,2023-11-15
1415603,NPI,1000,10.16,167,3526,1186447.51,1599,721.0,1875,2024-09-23
1426657,NPI,1000,6.02,286,1989,518363.32,1268,643.0,1863,2024-09-06
1166284,NPI,1000,9.28,44,510,10646.42,0,20.0,552,2023-12-06
1269875,NPI,1000,1.86,47,56,3025.7,3,54.0,112,2023-11-11
1269877,NPI,1000,2.49,52,83,6716.33,5,80.0,160,2023-11-14


In [0]:
daily_forecast = spark.read.table('default.daily_forecast_df').filter(F.col("Is_Predicted")== "Y")

In [0]:
monthly_window = (Window()
   .partitionBy(col('MATNR'),col('AREA'),col('LGORT'))
   .orderBy(col("ds").desc())
   .rowsBetween(0, 30))

In [0]:
day30_pred = daily_forecast.where(col('ds') < date_add(CURRENT_D,30)).groupBy("MATNR","AREA",'LGORT')\
                                    .agg(round(sum("yhat")).alias("Predicted_Demand_30_Day"))

Mat_stats = matnr_cons.withColumn("total",round(sum("y").over(monthly_window)))\
                    .withColumn("row_num", 
                                row_number().over(Window().partitionBy(col('MATNR'),col('AREA'),col('LGORT'))\
                    .orderBy(col('ds').desc())) % 30).filter(col('row_num') == 1)\
                    .groupby("MATNR","AREA","LGORT").agg(round(F.avg("total"),2).alias("Avg_Monthly_Demand"))\
                    .join(day30_pred,["MATNR","AREA","LGORT"],'inner')



In [0]:
dmd_lt = demand_stats.select('MATNR','AREA','LGORT','STDDEV_DEMAND')\
.join(Mat_stats.select('MATNR','AREA', 'LGORT', 'Avg_Monthly_Demand'), ['MATNR','AREA' ,'LGORT'], 'inner')\
.join(lead_time_stats,['MATNR','AREA', 'LGORT'],'inner').withColumnRenamed('Avg_Monthly_Demand', 'AVG_DEMAND')\
.withColumnRenamed('Avg_Monthly_Bo_Demand', 'BO_AVG_DEMAND')

safety_stock_cal =  dmd_lt.withColumn('SAFETY_STOCK',
                                      round(z*col('STDDEV_DEMAND')*sqrt(col('MEDIAN_LEADTIME')), 0))\
                        .select('MATNR','AREA', 'LGORT','SAFETY_STOCK')

In [0]:
#Median leadtime
windowSpec = Window.partitionBy(['MATNR',"AREA","LGORT"]).orderBy("ds")
prediction_and_leadtime = daily_forecast.join(lead_time_stats, on = ['MATNR','AREA',"LGORT"], how='inner')

median_leadtime = prediction_and_leadtime.withColumn("row_no", F.row_number().over(windowSpec))\
                        .filter(F.col("row_no") <= F.col("MEDIAN_LEADTIME"))


median_leadtime_min_inv = median_leadtime\
                        .groupby("MATNR","AREA","LGORT").agg(F.round((F.sum(F.col("yhat")))).alias("MINIMUM_INV"))\
                        .join(safety_stock_cal,["MATNR","AREA","LGORT"],"left")\
                          .withColumn("MINIMUM_INV",F.col("MINIMUM_INV") + F.col("Safety_Stock"))\
                        .drop("Safety_Stock")


In [0]:
BACKORDER = BO_Details.groupby('MATNR','AREA').agg(sum('QTY').alias('Bo_Qty'))

In [0]:
final_data = demand_over_leadtime_selected.join(BACKORDER,['MATNR','AREA'],'left')\
                          .join(median_leadtime_min_inv\
                           .select("MATNR","AREA","LGORT" ,"MINIMUM_INV"), 
                                                               ["MATNR","AREA","LGORT"], "inner")\
                           .join(Mat_stats.select("MATNR","AREA","LGORT","Avg_Monthly_Demand",
                                             "Predicted_Demand_30_Day"),["MATNR","AREA","LGORT"], "inner")\
                           .drop("MINIMUM_INV")



In [0]:
reorder_date_calc = final_data.select('MATNR','AREA',"LGORT",'Inventory','Open_POs','Reorder_Point','Bo_Qty')\
                .withColumn("incoming", (F.col("Inventory") + F.col("Open_POs")) - col('Bo_Qty') )\
               .withColumn("extra_items", F.when(F.col("incoming") > F.col("Reorder_Point"), 
                                                 F.col("incoming") - F.col("Reorder_Point")).otherwise(0))\
               .select('MATNR','AREA',"LGORT",'extra_items').join(daily_forecast, ['MATNR','AREA',"LGORT"], 'inner')

cumSumPartition = Window.partitionBy(['MATNR','AREA',"LGORT"]).orderBy("ds").rowsBetween(-sys.maxsize -1,0)

reorder_date_calc= reorder_date_calc.filter(F.col('ds')>=CURRENT_D).withColumn("cum_pred",
                                   F.sum(reorder_date_calc.yhat).over(cumSumPartition)).filter(F.col("extra_items") <= F.col("cum_pred"))\
                                    .groupby("MATNR","AREA","LGORT").agg(F.min("ds").alias("Reorder_dates"))\
                                       .withColumn('Reorder_dates', F.to_date('Reorder_dates'))

final_data = final_data.join(reorder_date_calc,['MATNR','AREA',"LGORT"],'left')
display(final_data)

MATNR,AREA,LGORT,demand_per_day,MEDIAN_LEADTIME,Inventory,INV_CAPEX,Open_POs,Unit_Price,Reorder_Point,Inventory_Exhaust_Date,Bo_Qty,Avg_Monthly_Demand,Predicted_Demand_30_Day,Reorder_dates
1166284,NPI,1000,9.28,44,510,10646.42,0,20.0,552,2023-12-06,null,290.06,430.0,2023-10-12
1223406,NPI,1000,1.2,144,41,733.0799999999999,251,16.16,206,2023-11-15,null,37.19,80.0,2023-10-12
1269875,NPI,1000,1.86,47,56,3025.7,3,54.0,112,2023-11-11,null,57.46,74.0,2023-10-12
1269877,NPI,1000,2.49,52,83,6716.33,5,80.0,160,2023-11-14,null,77.2,97.0,2023-10-12
1269878,NPI,1000,2.55,49,0,0.0,0,80.0,156,2023-10-12,1,79.15,115.0,2023-10-12
1415244,NPI,1000,2.88,182,32,12566.4,122,341.0,589,2023-10-23,null,88.96,117.0,2023-10-12
1415319,NPI,1000,8.51,176,189,9792.09,157,49.0,1686,2023-11-03,null,263.41,352.0,2023-10-12
1415603,NPI,1000,10.16,167,3526,1186447.51,1599,721.0,1875,2024-09-23,null,314.09,455.0,2023-10-12
1426657,NPI,1000,6.02,286,1989,518363.32,1268,643.0,1863,2024-09-06,null,186.11,253.0,2023-10-12
1426745,NPI,1000,10.16,181,3690,88630.54,1610,52.0,2021,2024-10-09,null,313.83,471.0,2023-10-12


In [0]:
final_data = final_data.withColumn("incoming", F.col("Inventory") + F.col("Open_POs"))\
                        .withColumn("today", CURRENT_D)\
                        .withColumn("extra_items", F.when(F.col("incoming") > F.col("Reorder_Point"),
                                                          F.col("incoming") - F.col("Reorder_Point")).otherwise(0))\
                        .withColumn("extra_days", F.floor(F.col("extra_items")/F.col("demand_per_day")).cast(T.IntegerType()))\
                        .withColumn("Reorder_Date", F.when((F.col('Reorder_dates').isNull()) | (F.col('extra_items') ==      
                                                  0),F.expr("date_add(today, extra_days)"))\
                                    .otherwise(F.col('Reorder_dates')))\
                        .withColumn("Quantity_to_Order", F.col("Reorder_Point") - F.col("incoming") + 
                                                                F.col("Predicted_Demand_30_Day"))\
                        .withColumn("Quantity_to_Order", when(col('Quantity_to_Order')<0,lit(0))\
                                                               .otherwise(col('Quantity_to_Order')))\
                        .withColumn("Predicted_CapEx", 
                                    round(F.col("Quantity_to_Order") * F.col("Unit_Price"),2))\
                        .withColumn("Inventory_CapEx", col('INV_CAPEX').cast(T.DecimalType(29, 2)))\
                        .withColumn("UnderStock", F.when((F.col("incoming") <= F.col("Reorder_Point")), True).otherwise(False))\
                        .withColumn("OverStock", F.when((F.col("incoming") > 2*F.col("Reorder_Point")) & 
                                                        (F.col("inventory") != 0), True).otherwise(False))\
                        .withColumn("OverStock", F.when(col('OverStock').isNotNull(), col('OverStock')).otherwise(False))\
                        .withColumn("UnderStock", F.when(col('UnderStock').isNotNull(), col('UnderStock')).otherwise(False))\
                        .withColumn("UnderStock_percent", F.when((F.col("UnderStock") == 1), 
                               F.round(((F.col("Reorder_Point")-F.col("incoming"))/F.col("Reorder_Point")) * 100, 2)).otherwise(0))\
                        .withColumn("OverStock_percent", F.when((F.col("OverStock") == 1), 
                               F.round(((F.col("incoming")-2*F.col("Reorder_Point"))/F.col("Reorder_Point")) * 100, 2)).otherwise(0))\
                        .withColumn("OverStock_Items",F.when(F.col("OverStock") == 1, 
                                                              F.col("incoming") - 2*F.col("Reorder_Point")).otherwise(0))\
                        .withColumn("OverStock_CapEx", F.col("OverStock_Items") * F.col("Unit_Price"))\
                        .join(safety_stock_cal,["MATNR","AREA","LGORT"],"inner")\
                        .withColumn("Safety_Stock",F.col("Safety_Stock").cast("Long")).withColumn("Date", CURRENT_D)\
                        .withColumn("Avg_Monthly_Demand",F.col("Avg_Monthly_Demand").cast("float"))\
                          .fillna(value=0,subset=["OverStock_percent",'UnderStock_percent','Bo_Qty'])\
                        .drop( 'Reorder_dates', "today", "INV_CAPEX","incoming","extra_items","extra_days")

display(final_data) 

MATNR,AREA,LGORT,demand_per_day,MEDIAN_LEADTIME,Inventory,Open_POs,Unit_Price,Reorder_Point,Inventory_Exhaust_Date,Bo_Qty,Avg_Monthly_Demand,Predicted_Demand_30_Day,Reorder_Date,Quantity_to_Order,Predicted_CapEx,Inventory_CapEx,UnderStock,OverStock,UnderStock_percent,OverStock_percent,OverStock_Items,OverStock_CapEx,Safety_Stock,Date
1166284,NPI,1000,9.28,44,510,0,20.0,552,2023-12-06,0,290.06,430.0,2023-10-12,472.0,9440.0,10646.42,true,false,7.61,0.0,0,0.0,143,2023-10-12
1223406,NPI,1000,1.2,144,41,251,16.16,206,2023-11-15,0,37.19,80.0,2023-10-12,0.0,0.0,733.08,false,false,0.0,0.0,0,0.0,33,2023-10-12
1269875,NPI,1000,1.86,47,56,3,54.0,112,2023-11-11,0,57.46,74.0,2023-10-12,127.0,6858.0,3025.7,true,false,47.32,0.0,0,0.0,24,2023-10-12
1269877,NPI,1000,2.49,52,83,5,80.0,160,2023-11-14,0,77.2,97.0,2023-10-12,169.0,13520.0,6716.33,true,false,45.0,0.0,0,0.0,30,2023-10-12
1269878,NPI,1000,2.55,49,0,0,80.0,156,2023-10-12,1,79.15,115.0,2023-10-12,271.0,21680.0,0.0,true,false,100.0,0.0,0,0.0,30,2023-10-12
1415244,NPI,1000,2.88,182,32,122,341.0,589,2023-10-23,0,88.96,117.0,2023-10-12,552.0,188232.0,12566.4,true,false,73.85,0.0,0,0.0,64,2023-10-12
1415319,NPI,1000,8.51,176,189,157,49.0,1686,2023-11-03,0,263.41,352.0,2023-10-12,1692.0,82908.0,9792.09,true,false,79.48,0.0,0,0.0,187,2023-10-12
1415603,NPI,1000,10.16,167,3526,1599,721.0,1875,2024-09-23,0,314.09,455.0,2023-10-12,0.0,0.0,1186447.51,false,true,0.0,73.33,1375,991375.0,178,2023-10-12
1426657,NPI,1000,6.02,286,1989,1268,643.0,1863,2024-09-06,0,186.11,253.0,2023-10-12,0.0,0.0,518363.32,false,false,0.0,0.0,0,0.0,140,2023-10-12
1426745,NPI,1000,10.16,181,3690,1610,52.0,2021,2024-10-09,0,313.83,471.0,2023-10-12,0.0,0.0,88630.54,false,true,0.0,62.25,1258,65416.0,182,2023-10-12


In [0]:
final_data.drop('UnderStock','OverStock','UnderStock_percent','OverStock_percent','OverStock_Items','OverStock_CapEx').write.format("delta").mode("overwrite").saveAsTable('default.Predicted_Order_Quantity')

In [0]:
final_data.select('MATNR','AREA','LGORT','UnderStock','OverStock','UnderStock_percent','OverStock_percent','OverStock_Items','OverStock_CapEx').write.format("delta").mode("overwrite").saveAsTable('default.Material_Stock_Details')